In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

def lstm_model(df):
    """
    Appliquer le modèle LSTM pour prédire la demande en produits sanguins.
    Le modèle est entraîné sur une séquence de 60 jours pour prédire la demande future.
    """
    # Mise à l'échelle des données
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_scaled = scaler.fit_transform(df[['demande']])

    # Création des séquences pour l'entraînement du modèle LSTM
    X_lstm, y_lstm = [], []
    for i in range(60, len(df_scaled)):
        X_lstm.append(df_scaled[i-60:i, 0])  # Séquence de 60 jours
        y_lstm.append(df_scaled[i, 0])  # Valeur suivante

    X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)
    X_lstm = X_lstm.reshape(X_lstm.shape[0], X_lstm.shape[1], 1)  # Remodeler pour LSTM

    # Définition du modèle LSTM
    lstm_model = tf.keras.Sequential([
        tf.keras.layers.LSTM(units=50, return_sequences=True, input_shape=(X_lstm.shape[1], 1)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(units=50, return_sequences=False),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=1)
    ])

    # Compilation et entraînement du modèle
    lstm_model.compile(optimizer='adam', loss='mean_squared_error')
    lstm_model.fit(X_lstm, y_lstm, epochs=10, batch_size=32)

    # Prédictions avec le modèle LSTM
    predictions_lstm = lstm_model.predict(X_lstm)

    # Visualisation des résultats
    plt.figure(figsize=(10, 6))
    plt.plot(df.index[60:], y_lstm, label='Demande réelle', color='blue')
    plt.plot(df.index[60:], predictions_lstm, label='Prévisions LSTM', color='green')
    plt.title('Comparaison des prévisions LSTM')
    plt.xlabel('Date')
    plt.ylabel('Quantité demandée')
    plt.legend()
    plt.tight_layout()
    plt.show()

    return predictions_lstm

# Charger les données prétraitées et appliquer le modèle LSTM
df = pd.read_csv('results/preprocessed_data.csv')
predictions_lstm = lstm_model(df)
